In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install matplotlib
!pip install sklearn
!pip install folium
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: - 
  - anaconda/linux-64::ca-certificates-2020.10.14-0, anaconda/linux-64::openssl-1.1.1h-h7b6447c_0
  - anaconda/linux-64::ca-certificates-2020.10.14-0, defaults/linux-64::openssl-1.1.1h-h7b6447c_0
  - anaconda/linux-64::openssl-1.1.1h-h7b6447c_0, defaults/linux-64::ca-certificates-2020.10.14-0
  - defaults/linux-64::ca-certificates-2020.10.14-0, defaults/linux-64::openssl-1.1.1h-h7b6447cdone

# All requested packages already installed.

Libraries imported.


#### By Merging the dataframes I will be able to get the intersection of both dataframes 
##### **Pandas Docs:** Column or index level names to join on. These must be found in both DataFrames. If on is None and not merging on indexes then this defaults to the intersection of the columns in both DataFrames.


In [2]:
df_pc = pd.read_csv('https://raw.githubusercontent.com/smartlearningci/Coursera_Capstone/main/Geospatial_Coordinates.csv')
#df_pc = pd.read_csv('/home/eduardo/Documentos/Alunos/IBM/coursera/Capstone/Geospatial_Coordinates.csv')
df_wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df_toronto = df_pc.merge(df_wiki,on='Postal Code')

In [3]:
df_toronto_neighb = df_toronto.drop(columns=['Latitude','Longitude'])

In [4]:
df_toronto_neighb.shape

(103, 3)

In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [5]:
df_toronto = df_toronto[['Postal Code','Borough','Neighbourhood','Latitude','Longitude']]

In [6]:
address = 'Toronto Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [7]:
# create map of Toronot using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [8]:
CLIENT_ID = '3ID4ZDQWD3AA4ECXB11OU0AW5PD0AZ32EXTBDWF3R5SNALWZ' # your Foursquare ID
CLIENT_SECRET = 'VJHPDGGLLCU1T3AK1151MTKG0RKPJB0UN3GKJYRF4BS1QIN5' # your Foursquare Secret
ACCESS_TOKEN = '1DUME1SSENWRKHYMDELF12NQA5XBMRGKCR52FYYESPGZW20D' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

print(df_toronto['Neighbourhood'][0])


My credentails:
CLIENT_ID: 3ID4ZDQWD3AA4ECXB11OU0AW5PD0AZ32EXTBDWF3R5SNALWZ
CLIENT_SECRET:VJHPDGGLLCU1T3AK1151MTKG0RKPJB0UN3GKJYRF4BS1QIN5
Malvern, Rouge


In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_data = df_toronto.reset_index(drop=True)

toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [12]:
print(toronto_venues.shape)
toronto_venues.head()

(2107, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [13]:
#Exploring the information retrieved
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,54,54,54,54,54,54
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18


In [14]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 266 uniques categories.


In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

col_num = toronto_onehot.columns.get_loc('Neighborhood')
# move neighborhood column to the first column
fixed_columns = ['Neighborhood'] + list(toronto_onehot.columns[:col_num ])+ list(toronto_onehot.columns[col_num+1:])

toronto_onehot = toronto_onehot[fixed_columns]


In [19]:
n = len(toronto_onehot)
for i in range(n):
    taux = toronto_onehot.iloc[i][1:]
    #print(toronto_onehot.iloc[i][0] + " " + " " + str(taux[taux > 0]))


In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Clothing Store,Skating Rink,Breakfast Spot
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Athletics & Sports,Pharmacy,Pub
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Shopping Mall,Diner,Supermarket
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Thai Restaurant,Juice Bar


In [24]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [26]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,Print Shop,Yoga Studio,Dog Run,Dessert Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Construction & Landscaping,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Restaurant,Medical Center,Rental Car Location,Intersection,Bank
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Soccer Field,Pharmacy,Korean BBQ Restaurant,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Lounge,Gas Station,Bakery,Bank,Athletics & Sports


In [27]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    if (np.isnan(cluster) == False):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]

,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,-79.239476,0.0,Playground,Yoga Studio,Dog Run,Department Store,Dessert Shop
14,-79.284577,0.0,Park,Intersection,Playground,Dog Run,Dessert Shop


In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]

,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,-79.194353,1.0,Fast Food Restaurant,Print Shop,Yoga Studio,Dog Run,Dessert Shop
1,-79.160497,1.0,Construction & Landscaping,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant
2,-79.188711,1.0,Restaurant,Medical Center,Rental Car Location,Intersection,Bank
3,-79.216917,1.0,Coffee Shop,Soccer Field,Pharmacy,Korean BBQ Restaurant,Dog Run
4,-79.239476,1.0,Lounge,Gas Station,Bakery,Bank,Athletics & Sports
6,-79.262029,1.0,Hobby Shop,Coffee Shop,Discount Store,Department Store,Electronics Store
7,-79.284577,1.0,Bakery,Ice Cream Shop,Bus Station,Park,Metro Station
8,-79.239476,1.0,American Restaurant,Motel,Skating Rink,Dog Run,Dessert Shop
9,-79.264848,1.0,General Entertainment,College Stadium,Café,Skating Rink,Doner Restaurant
10,-79.273304,1.0,Indian Restaurant,Pet Store,Light Rail Station,Vietnamese Restaurant,Gaming Cafe


In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]

,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
94,-79.554724,2.0,Bakery,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store


In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]

,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
90,-79.506944,3.0,River,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]

,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
20,-79.374714,4.0,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant
21,-79.408493,4.0,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant
23,-79.400049,4.0,Convenience Store,Park,Yoga Studio,Dim Sum Restaurant,Diner
25,-79.329656,4.0,Park,Food & Drink Shop,Yoga Studio,Dog Run,Dessert Shop
40,-79.338106,4.0,Convenience Store,Park,Yoga Studio,Dim Sum Restaurant,Diner
48,-79.383160,4.0,Trail,Park,Yoga Studio,Department Store,Dessert Shop
50,-79.377529,4.0,Park,Trail,Playground,Deli / Bodega,Department Store
74,-79.453512,4.0,Park,Women's Store,Pool,Dog Run,Department Store
